In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings() #모든 문서를 벡터로 변경

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings) #벡터들을 저장하는 변수 vectorstore

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
])

chain = retriever | 

chain.run("Describe Victory Mansions")

'Victory Mansions is a building where Winston Smith lives. It is a place with glass doors, a smelly hallway of boiled cabbage and old rag mats, and a seven-flight walk-up due to the non-functioning lift. Inside the flat, there is a telescreen that cannot be completely shut off and a poster with the face of a man, emphasizing the theme that "BIG BROTHER IS WATCHING YOU." Winston\'s flat in Victory Mansions is where he interacts with the telescreen and experiences the oppressive atmosphere of the Party\'s surveillance.'